## List deployed SPSS models on test environment

**Note:** Server is in `9.30.*.*` network.

In [1]:
import requests
import json
import base64
import json

username = 'testuser'
password = 'TestMe'
basic_token = base64.b64encode('{}:{}'.format(username, password).encode('utf-8')).decode()
print(basic_token)

dGVzdHVzZXI6VGVzdE1l


In [2]:
service_url = 'http://9.30.219.197:9080'
config_endpoint = 'scoring/rest/configuration'
service_header = {"Authorization": " ".join(["Basic", basic_token])}

## List models with configured scoring 

In [3]:
resp_config = requests.get('/'.join([service_url, config_endpoint]), headers=service_header)

if resp_config.status_code == 200:
    config_meta = resp_config.json()
    print(json.dumps(config_meta, indent=2))
else:
    print(resp_config)

[
  {
    "cfgSerial": "091edbc5c4e74be500000167bc5c18e058c9",
    "state": "ACTIVE",
    "cacheSize": 1,
    "id": "credit_risk_lr",
    "modelReference": {
      "label": "LATEST",
      "resourcePath": "/Test/credit_risk_logistic.str",
      "id": "091edbc5c4e74be500000167bc5c18e058bf"
    },
    "configurationStatus": {
      "statusCode": "INFORMATION",
      "message": "Started"
    }
  },
  {
    "cfgSerial": "091edbc5418240f600000166f5190140dac6",
    "state": "SUSPENDED",
    "cacheSize": 1,
    "aliasTo": "091edbc5418240f600000166f5190140daaa",
    "id": "drug_neural_network",
    "modelReference": {
      "label": "LATEST",
      "resourcePath": "/Test/Drug1n_new.str",
      "id": "091edbc5418240f600000166f5190140da96"
    },
    "configurationStatus": {
      "statusCode": "INFORMATION",
      "message": "Suspended"
    }
  },
  {
    "cfgSerial": "091edbc5c4e74be5000001686a7c7110a1bd",
    "state": "ACTIVE",
    "cacheSize": 1,
    "id": "ai_credit_risk_blogistic",
    "mo

## Credit risk model

### Binary classification using boosted trees

In [4]:
[model_config] = [cm for cm in config_meta if cm['id'] == 'ai_credit_risk_btree']
print(json.dumps(model_config, indent=2))

{
  "cfgSerial": "091edbc5c4e74be5000001686a7c71109992",
  "state": "ACTIVE",
  "cacheSize": 1,
  "id": "ai_credit_risk_btree",
  "modelReference": {
    "label": "LATEST",
    "resourcePath": "/aiopenscale/credit_risk_btree.str",
    "id": "091edbc5c4e74be5000001686a7c71109872"
  },
  "configurationStatus": {
    "statusCode": "INFORMATION",
    "message": "Started"
  }
}


**List model metadata**

In [5]:
meta_endpoint = 'scoring/rest/configuration/{id}/metadata'.format(id=model_config['id'])
print(json.dumps(meta_endpoint, indent=2))

resp_meta = requests.get('/'.join([service_url, meta_endpoint]), headers=service_header)
if resp_meta.status_code == 200:
    model_meta = resp_meta.json()
    print(json.dumps(model_meta, indent=2))
else:
    print(resp_meta)

"scoring/rest/configuration/ai_credit_risk_btree/metadata"
{
  "metadataInputTable": [
    {
      "id": "id17STY5BAZ1Y",
      "name": "credit_risk_training.csv",
      "metadataInputField": [
        {
          "name": "CheckingStatus",
          "type": "string",
          "description": "CheckingStatus",
          "isRequired": true
        },
        {
          "name": "LoanDuration",
          "type": "long",
          "description": "LoanDuration",
          "isRequired": true
        },
        {
          "name": "CreditHistory",
          "type": "string",
          "description": "CreditHistory",
          "isRequired": true
        },
        {
          "name": "LoanPurpose",
          "type": "string",
          "description": "LoanPurpose",
          "isRequired": true
        },
        {
          "name": "LoanAmount",
          "type": "long",
          "description": "LoanAmount",
          "isRequired": true
        },
        {
          "name": "ExistingSavings"

**Score credit risk model**

In [6]:
[model_meta_InputTable] = model_meta['metadataInputTable']
print(json.dumps(model_meta_InputTable, indent=2))

# Input tables
print('\n',[{'id': x['id'], 'name': x['name']} for x in model_meta['metadataInputTable']])

# First input table field names
print('\n',[fld['name'] for fld in model_meta['metadataInputTable'][0]['metadataInputField']])

{
  "id": "id17STY5BAZ1Y",
  "name": "credit_risk_training.csv",
  "metadataInputField": [
    {
      "name": "CheckingStatus",
      "type": "string",
      "description": "CheckingStatus",
      "isRequired": true
    },
    {
      "name": "LoanDuration",
      "type": "long",
      "description": "LoanDuration",
      "isRequired": true
    },
    {
      "name": "CreditHistory",
      "type": "string",
      "description": "CreditHistory",
      "isRequired": true
    },
    {
      "name": "LoanPurpose",
      "type": "string",
      "description": "LoanPurpose",
      "isRequired": true
    },
    {
      "name": "LoanAmount",
      "type": "long",
      "description": "LoanAmount",
      "isRequired": true
    },
    {
      "name": "ExistingSavings",
      "type": "string",
      "description": "ExistingSavings",
      "isRequired": true
    },
    {
      "name": "EmploymentDuration",
      "type": "string",
      "description": "EmploymentDuration",
      "isRequired": true

In [7]:
scoring_endpoint = 'scoring/rest/configuration/{id}/score'.format(id=model_config['id'])
scoring_payload = {"id": model_config['id'], "requestInputTable": [{"id": model_meta_InputTable["id"],
                                                    "requestInputRow": [{"input": [{"name": "CheckingStatus", "value": "0_to_200"},
                                                                                   {"name": "LoanDuration", "value": "31"},
                                                                                   {"name": "CreditHistory", "value": "credits_paid_to_date"},
                                                                                   {"name": "LoanPurpose", "value": "other"},
                                                                                   {"name": "LoanAmount", "value": "1889"},
                                                                                   {"name": "ExistingSavings", "value": "100_to_500"},
                                                                                   {"name": "EmploymentDuration", "value": "less_1"},
                                                                                   {"name": "InstallmentPercent", "value": "3"},
                                                                                   {"name": "Sex", "value": "female"},
                                                                                   {"name": "OthersOnLoan", "value": "none"},
                                                                                   {"name": "CurrentResidenceDuration", "value": "3"},
                                                                                   {"name": "OwnsProperty", "value": "savings_insurance"},
                                                                                   {"name": "Age", "value": "32"},
                                                                                   {"name": "InstallmentPlans", "value": "none"},
                                                                                   {"name": "Housing", "value": "own"},
                                                                                   {"name": "ExistingCreditsCount", "value": "1"},
                                                                                   {"name": "Job", "value": "skilled"},
                                                                                   {"name": "Dependents", "value": "1"},
                                                                                   {"name": "Telephone", "value": "none"},
                                                                                   {"name": "ForeignWorker", "value": "yes"},
                                                                                   {"name": "Risk", "value": ""}
                                                                                  ]}]}]}

In [8]:
resp_score = requests.post('/'.join([service_url, scoring_endpoint]), json = scoring_payload, headers=service_header)
scoring_result = resp_score.json()
print(json.dumps(scoring_result, indent=2))

{
  "providedBy": "ai_credit_risk_btree",
  "id": "478be182-00c8-4d17-a365-7a3f7d48e63a",
  "columnNames": {
    "name": [
      "CheckingStatus",
      "LoanDuration",
      "CreditHistory",
      "LoanPurpose",
      "LoanAmount",
      "ExistingSavings",
      "EmploymentDuration",
      "InstallmentPercent",
      "Sex",
      "OthersOnLoan",
      "CurrentResidenceDuration",
      "OwnsProperty",
      "Age",
      "InstallmentPlans",
      "Housing",
      "ExistingCreditsCount",
      "Job",
      "Dependents",
      "Telephone",
      "ForeignWorker",
      "$C-Risk",
      "$CC-Risk"
    ]
  },
  "rowValues": [
    {
      "value": [
        {
          "value": "0_to_200"
        },
        {
          "value": "31"
        },
        {
          "value": "credits_paid_to_date"
        },
        {
          "value": "other"
        },
        {
          "value": "1889"
        },
        {
          "value": "100_to_500"
        },
        {
          "value": "less_1"
    

### Binary classification using multinomial logistic

In [9]:
[model_config] = [cm for cm in config_meta if cm['id'] == 'ai_credit_risk_mlogistic']
print(json.dumps(model_config, indent=2))

{
  "cfgSerial": "091edbc5c4e74be5000001686a7c71109a8a",
  "state": "ACTIVE",
  "cacheSize": 1,
  "id": "ai_credit_risk_mlogistic",
  "modelReference": {
    "label": "LATEST",
    "resourcePath": "/aiopenscale/credit_risk_mlogistic.str",
    "id": "091edbc5c4e74be5000001686a7c7110990e"
  },
  "configurationStatus": {
    "statusCode": "INFORMATION",
    "message": "Started"
  }
}


**List model metadata**

In [10]:
meta_endpoint = 'scoring/rest/configuration/{id}/metadata'.format(id=model_config['id'])
print(json.dumps(meta_endpoint, indent=2))

resp_meta = requests.get('/'.join([service_url, meta_endpoint]), headers=service_header)
model_meta = resp_meta.json()
print(json.dumps(model_meta, indent=2))

"scoring/rest/configuration/ai_credit_risk_mlogistic/metadata"
{
  "metadataInputTable": [
    {
      "id": "id17STY5BAZ1Y",
      "name": "credit_risk_training.csv",
      "metadataInputField": [
        {
          "name": "CheckingStatus",
          "type": "string",
          "description": "CheckingStatus",
          "isRequired": true
        },
        {
          "name": "LoanDuration",
          "type": "long",
          "description": "LoanDuration",
          "isRequired": true
        },
        {
          "name": "CreditHistory",
          "type": "string",
          "description": "CreditHistory",
          "isRequired": true
        },
        {
          "name": "LoanPurpose",
          "type": "string",
          "description": "LoanPurpose",
          "isRequired": true
        },
        {
          "name": "LoanAmount",
          "type": "long",
          "description": "LoanAmount",
          "isRequired": true
        },
        {
          "name": "ExistingSavi

**Score credit risk model**

In [11]:
[model_meta_InputTable] = model_meta['metadataInputTable']
print(json.dumps(model_meta_InputTable, indent=2))

# Input tables
print('\n',[{'id': x['id'], 'name': x['name']} for x in model_meta['metadataInputTable']])

# First input table field names
print('\n', [fld['name'] for fld in model_meta['metadataInputTable'][0]['metadataInputField']])

{
  "id": "id17STY5BAZ1Y",
  "name": "credit_risk_training.csv",
  "metadataInputField": [
    {
      "name": "CheckingStatus",
      "type": "string",
      "description": "CheckingStatus",
      "isRequired": true
    },
    {
      "name": "LoanDuration",
      "type": "long",
      "description": "LoanDuration",
      "isRequired": true
    },
    {
      "name": "CreditHistory",
      "type": "string",
      "description": "CreditHistory",
      "isRequired": true
    },
    {
      "name": "LoanPurpose",
      "type": "string",
      "description": "LoanPurpose",
      "isRequired": true
    },
    {
      "name": "LoanAmount",
      "type": "long",
      "description": "LoanAmount",
      "isRequired": true
    },
    {
      "name": "ExistingSavings",
      "type": "string",
      "description": "ExistingSavings",
      "isRequired": true
    },
    {
      "name": "EmploymentDuration",
      "type": "string",
      "description": "EmploymentDuration",
      "isRequired": true

In [12]:
scoring_endpoint = 'scoring/rest/configuration/{id}/score'.format(id=model_config['id'])
scoring_payload = {"id": model_config['id'], "requestInputTable": [{"id": model_meta_InputTable["id"],
                                                    "requestInputRow": [{"input": [{"name": "CheckingStatus", "value": "0_to_200"},
                                                                                   {"name": "LoanDuration", "value": "31"},
                                                                                   {"name": "CreditHistory", "value": "credits_paid_to_date"},
                                                                                   {"name": "LoanPurpose", "value": "other"},
                                                                                   {"name": "LoanAmount", "value": "1889"},
                                                                                   {"name": "ExistingSavings", "value": "100_to_500"},
                                                                                   {"name": "EmploymentDuration", "value": "less_1"},
                                                                                   {"name": "InstallmentPercent", "value": "3"},
                                                                                   {"name": "Sex", "value": "female"},
                                                                                   {"name": "OthersOnLoan", "value": "none"},
                                                                                   {"name": "CurrentResidenceDuration", "value": "3"},
                                                                                   {"name": "OwnsProperty", "value": "savings_insurance"},
                                                                                   {"name": "Age", "value": "32"},
                                                                                   {"name": "InstallmentPlans", "value": "none"},
                                                                                   {"name": "Housing", "value": "own"},
                                                                                   {"name": "ExistingCreditsCount", "value": "1"},
                                                                                   {"name": "Job", "value": "skilled"},
                                                                                   {"name": "Dependents", "value": "1"},
                                                                                   {"name": "Telephone", "value": "none"},
                                                                                   {"name": "ForeignWorker", "value": "yes"},
                                                                                   {"name": "Risk", "value": ""}
                                                                                  ]}]}]}

In [13]:
resp_score = requests.post('/'.join([service_url, scoring_endpoint]), json = scoring_payload, headers=service_header)
scoring_result = resp_score.json()
print(json.dumps(scoring_result, indent=2))

{
  "providedBy": "ai_credit_risk_mlogistic",
  "id": "c22fbe17-2e9c-4b12-a473-b1ee583f8e58",
  "columnNames": {
    "name": [
      "CheckingStatus",
      "LoanDuration",
      "CreditHistory",
      "LoanPurpose",
      "LoanAmount",
      "ExistingSavings",
      "EmploymentDuration",
      "InstallmentPercent",
      "Sex",
      "OthersOnLoan",
      "CurrentResidenceDuration",
      "OwnsProperty",
      "Age",
      "InstallmentPlans",
      "Housing",
      "ExistingCreditsCount",
      "Job",
      "Dependents",
      "Telephone",
      "ForeignWorker",
      "Risk",
      "$L-Risk",
      "$LP-Risk",
      "$LRP-Risk"
    ]
  },
  "rowValues": [
    {
      "value": [
        {
          "value": "0_to_200"
        },
        {
          "value": "31"
        },
        {
          "value": "credits_paid_to_date"
        },
        {
          "value": "other"
        },
        {
          "value": "1889"
        },
        {
          "value": "100_to_500"
        },
     

### Binary classification using binomial logistic

In [14]:
[model_config] = [cm for cm in config_meta if cm['id'] == 'ai_credit_risk_blogistic']
print(json.dumps(model_config, indent=2))

{
  "cfgSerial": "091edbc5c4e74be5000001686a7c7110a1bd",
  "state": "ACTIVE",
  "cacheSize": 1,
  "id": "ai_credit_risk_blogistic",
  "modelReference": {
    "label": "LATEST",
    "resourcePath": "/aiopenscale/credit_risk_blogistic.str",
    "id": "091edbc5c4e74be5000001686a7c7110a1b1"
  },
  "configurationStatus": {
    "statusCode": "INFORMATION",
    "message": "Started"
  }
}


**List model metadata**

In [15]:
meta_endpoint = 'scoring/rest/configuration/{id}/metadata'.format(id=model_config['id'])
print(json.dumps(meta_endpoint, indent=2))

resp_meta = requests.get('/'.join([service_url, meta_endpoint]), headers=service_header)
model_meta = resp_meta.json()
print(json.dumps(model_meta, indent=2))

"scoring/rest/configuration/ai_credit_risk_blogistic/metadata"
{
  "metadataInputTable": [
    {
      "id": "id17STY5BAZ1Y",
      "name": "credit_risk_training.csv",
      "metadataInputField": [
        {
          "name": "CheckingStatus",
          "type": "string",
          "description": "CheckingStatus",
          "isRequired": true
        },
        {
          "name": "LoanDuration",
          "type": "long",
          "description": "LoanDuration",
          "isRequired": true
        },
        {
          "name": "CreditHistory",
          "type": "string",
          "description": "CreditHistory",
          "isRequired": true
        },
        {
          "name": "LoanPurpose",
          "type": "string",
          "description": "LoanPurpose",
          "isRequired": true
        },
        {
          "name": "LoanAmount",
          "type": "long",
          "description": "LoanAmount",
          "isRequired": true
        },
        {
          "name": "ExistingSavi

**Score credit risk model**

In [16]:
[model_meta_InputTable] = model_meta['metadataInputTable']
print(json.dumps(model_meta_InputTable, indent=2))

# Input tables
print('\n', [{'id': x['id'], 'name': x['name']} for x in model_meta['metadataInputTable']])

# First input table field names
print('\n', [fld['name'] for fld in model_meta['metadataInputTable'][0]['metadataInputField']])

{
  "id": "id17STY5BAZ1Y",
  "name": "credit_risk_training.csv",
  "metadataInputField": [
    {
      "name": "CheckingStatus",
      "type": "string",
      "description": "CheckingStatus",
      "isRequired": true
    },
    {
      "name": "LoanDuration",
      "type": "long",
      "description": "LoanDuration",
      "isRequired": true
    },
    {
      "name": "CreditHistory",
      "type": "string",
      "description": "CreditHistory",
      "isRequired": true
    },
    {
      "name": "LoanPurpose",
      "type": "string",
      "description": "LoanPurpose",
      "isRequired": true
    },
    {
      "name": "LoanAmount",
      "type": "long",
      "description": "LoanAmount",
      "isRequired": true
    },
    {
      "name": "ExistingSavings",
      "type": "string",
      "description": "ExistingSavings",
      "isRequired": true
    },
    {
      "name": "EmploymentDuration",
      "type": "string",
      "description": "EmploymentDuration",
      "isRequired": true

In [17]:
scoring_endpoint = 'scoring/rest/configuration/{id}/score'.format(id=model_config['id'])
scoring_payload = {"id": model_config['id'], "requestInputTable": [{"id": model_meta_InputTable["id"],
                                                    "requestInputRow": [{"input": [{"name": "CheckingStatus", "value": "0_to_200"},
                                                                                   {"name": "LoanDuration", "value": "31"},
                                                                                   {"name": "CreditHistory", "value": "credits_paid_to_date"},
                                                                                   {"name": "LoanPurpose", "value": "other"},
                                                                                   {"name": "LoanAmount", "value": "1889"},
                                                                                   {"name": "ExistingSavings", "value": "100_to_500"},
                                                                                   {"name": "EmploymentDuration", "value": "less_1"},
                                                                                   {"name": "InstallmentPercent", "value": "3"},
                                                                                   {"name": "Sex", "value": "female"},
                                                                                   {"name": "OthersOnLoan", "value": "none"},
                                                                                   {"name": "CurrentResidenceDuration", "value": "3"},
                                                                                   {"name": "OwnsProperty", "value": "savings_insurance"},
                                                                                   {"name": "Age", "value": "32"},
                                                                                   {"name": "InstallmentPlans", "value": "none"},
                                                                                   {"name": "Housing", "value": "own"},
                                                                                   {"name": "ExistingCreditsCount", "value": "1"},
                                                                                   {"name": "Job", "value": "skilled"},
                                                                                   {"name": "Dependents", "value": "1"},
                                                                                   {"name": "Telephone", "value": "none"},
                                                                                   {"name": "ForeignWorker", "value": "yes"},
                                                                                   {"name": "Risk", "value": ""}
                                                                                  ]}]}]}

In [18]:
resp_score = requests.post('/'.join([service_url, scoring_endpoint]), json = scoring_payload, headers=service_header)
scoring_result = resp_score.json()
print(json.dumps(scoring_result, indent=2))

{
  "providedBy": "ai_credit_risk_blogistic",
  "id": "ffa5cbfb-e60d-4a00-b936-a90f0aa76693",
  "columnNames": {
    "name": [
      "CheckingStatus",
      "LoanDuration",
      "CreditHistory",
      "LoanPurpose",
      "LoanAmount",
      "ExistingSavings",
      "EmploymentDuration",
      "InstallmentPercent",
      "Sex",
      "OthersOnLoan",
      "CurrentResidenceDuration",
      "OwnsProperty",
      "Age",
      "InstallmentPlans",
      "Housing",
      "ExistingCreditsCount",
      "Job",
      "Dependents",
      "Telephone",
      "ForeignWorker",
      "Risk",
      "$L-Risk",
      "$LC-Risk"
    ]
  },
  "rowValues": [
    {
      "value": [
        {
          "value": "0_to_200"
        },
        {
          "value": "31"
        },
        {
          "value": "credits_paid_to_date"
        },
        {
          "value": "other"
        },
        {
          "value": "1889"
        },
        {
          "value": "100_to_500"
        },
        {
          "val

## Heart drug selection model

**Multiclass classification using neural neuron perceptron** without class probabilities

In [19]:
[model_config] = [cm for cm in config_meta if cm['id'] == 'ai_drug_mlp']
print(json.dumps(model_config, indent=2))

{
  "cfgSerial": "091edbc5c4e74be5000001686a7c71109a92",
  "state": "ACTIVE",
  "cacheSize": 1,
  "id": "ai_drug_mlp",
  "modelReference": {
    "label": "LATEST",
    "resourcePath": "/aiopenscale/Drug1n.str",
    "id": "091edbc5c4e74be5000001686a7c7110992c"
  },
  "configurationStatus": {
    "statusCode": "INFORMATION",
    "message": "Started"
  }
}


**List model metadata**

In [20]:
meta_endpoint = 'scoring/rest/configuration/{id}/metadata'.format(id=model_config['id'])
print(json.dumps(meta_endpoint, indent=2))

"scoring/rest/configuration/ai_drug_mlp/metadata"


In [21]:
resp_meta = requests.get('/'.join([service_url, meta_endpoint]), headers=service_header)
model_meta = resp_meta.json()
print(json.dumps(model_meta, indent=2))

{
  "metadataInputTable": [
    {
      "id": "id1JRRSIWAHWX",
      "name": "scoreInput",
      "metadataInputField": [
        {
          "name": "Age",
          "type": "long",
          "description": "Age",
          "isRequired": true
        },
        {
          "name": "Sex",
          "type": "string",
          "description": "Sex",
          "isRequired": true
        },
        {
          "name": "BP",
          "type": "string",
          "description": "BP",
          "isRequired": true
        },
        {
          "name": "Cholesterol",
          "type": "string",
          "description": "Cholesterol",
          "isRequired": true
        },
        {
          "name": "Na",
          "type": "double",
          "description": "Na",
          "isRequired": true
        },
        {
          "name": "K",
          "type": "double",
          "description": "K",
          "isRequired": true
        }
      ]
    }
  ],
  "metadataOutputField": [
    {
      "name"

**Score heart drug model**

In [22]:
[model_meta_InputTable] = model_meta['metadataInputTable']
print(json.dumps(model_meta_InputTable, indent=2))

# Input tables
print('\n', [{'id': x['id'], 'name': x['name']} for x in model_meta['metadataInputTable']])

# First input table field names
print('\n', [fld['name'] for fld in model_meta['metadataInputTable'][0]['metadataInputField']])

{
  "id": "id1JRRSIWAHWX",
  "name": "scoreInput",
  "metadataInputField": [
    {
      "name": "Age",
      "type": "long",
      "description": "Age",
      "isRequired": true
    },
    {
      "name": "Sex",
      "type": "string",
      "description": "Sex",
      "isRequired": true
    },
    {
      "name": "BP",
      "type": "string",
      "description": "BP",
      "isRequired": true
    },
    {
      "name": "Cholesterol",
      "type": "string",
      "description": "Cholesterol",
      "isRequired": true
    },
    {
      "name": "Na",
      "type": "double",
      "description": "Na",
      "isRequired": true
    },
    {
      "name": "K",
      "type": "double",
      "description": "K",
      "isRequired": true
    }
  ]
}

 [{'id': 'id1JRRSIWAHWX', 'name': 'scoreInput'}]

 ['Age', 'Sex', 'BP', 'Cholesterol', 'Na', 'K']


In [23]:
scoring_endpoint = 'scoring/rest/configuration/{id}/score'.format(id=model_config['id'])
scoring_payload = {"id": model_config['id'], "requestInputTable": [{"id": model_meta_InputTable["id"],
                                                    "requestInputRow": [{"input": [{"name": "Age", "value": "23.0"},
                                                                                   {"name": "Sex", "value": "F"},
                                                                                   {"name": "BP", "value": "HIGH"},
                                                                                   {"name": "Cholesterol", "value": "HIGH"},
                                                                                   {"name": "Na", "value": "0.792535"},
                                                                                   {"name": "K", "value": "0.031258"}
                                                                                  ]}]}]}

In [24]:
resp_score = requests.post('/'.join([service_url, scoring_endpoint]), json = scoring_payload, headers=service_header)
scoring_result = resp_score.json()
print(json.dumps(scoring_result, indent=2))

{
  "providedBy": "ai_drug_mlp",
  "id": "a1f29abd-da70-47c5-a48c-ae3a858bfffe",
  "columnNames": {
    "name": [
      "Age",
      "Sex",
      "BP",
      "Cholesterol",
      "Na",
      "K",
      "$N-Drug",
      "$NC-Drug"
    ]
  },
  "rowValues": [
    {
      "value": [
        {
          "value": "23"
        },
        {
          "value": "F"
        },
        {
          "value": "HIGH"
        },
        {
          "value": "HIGH"
        },
        {
          "value": "0.792535"
        },
        {
          "value": "0.031258"
        },
        {
          "value": "drugY"
        },
        {
          "value": "0.9999929901781122"
        }
      ]
    }
  ]
}


**Multiclass classification using neural neuron perceptron** with class probabilities

In [25]:
[model_config] = [cm for cm in config_meta if cm['id'] == 'ai_drug_mlp_proba']
print(json.dumps(model_config, indent=2))

{
  "cfgSerial": "091edbc5c4e74be5000001686a7c7110a120",
  "state": "ACTIVE",
  "cacheSize": 1,
  "id": "ai_drug_mlp_proba",
  "modelReference": {
    "label": "LATEST",
    "resourcePath": "/aiopenscale/Drug1n_proba.str",
    "id": "091edbc5c4e74be5000001686a7c7110a105"
  },
  "configurationStatus": {
    "statusCode": "INFORMATION",
    "message": "Started"
  }
}


**List model metadata**

In [26]:
meta_endpoint = 'scoring/rest/configuration/{id}/metadata'.format(id=model_config['id'])
print(json.dumps(meta_endpoint, indent=2))

"scoring/rest/configuration/ai_drug_mlp_proba/metadata"


In [27]:
resp_meta = requests.get('/'.join([service_url, meta_endpoint]), headers=service_header)
model_meta = resp_meta.json()
print(json.dumps(model_meta, indent=2))

{
  "metadataInputTable": [
    {
      "id": "id1JRRSIWAHWX",
      "name": "scoreInput",
      "metadataInputField": [
        {
          "name": "Age",
          "type": "long",
          "description": "Age",
          "isRequired": true
        },
        {
          "name": "Sex",
          "type": "string",
          "description": "Sex",
          "isRequired": true
        },
        {
          "name": "BP",
          "type": "string",
          "description": "BP",
          "isRequired": true
        },
        {
          "name": "Cholesterol",
          "type": "string",
          "description": "Cholesterol",
          "isRequired": true
        },
        {
          "name": "Na",
          "type": "double",
          "description": "Na",
          "isRequired": true
        },
        {
          "name": "K",
          "type": "double",
          "description": "K",
          "isRequired": true
        }
      ]
    }
  ],
  "metadataOutputField": [
    {
      "name"

**Score heart drug model**

In [28]:
[model_meta_InputTable] = model_meta['metadataInputTable']
print(json.dumps(model_meta_InputTable, indent=2))

{
  "id": "id1JRRSIWAHWX",
  "name": "scoreInput",
  "metadataInputField": [
    {
      "name": "Age",
      "type": "long",
      "description": "Age",
      "isRequired": true
    },
    {
      "name": "Sex",
      "type": "string",
      "description": "Sex",
      "isRequired": true
    },
    {
      "name": "BP",
      "type": "string",
      "description": "BP",
      "isRequired": true
    },
    {
      "name": "Cholesterol",
      "type": "string",
      "description": "Cholesterol",
      "isRequired": true
    },
    {
      "name": "Na",
      "type": "double",
      "description": "Na",
      "isRequired": true
    },
    {
      "name": "K",
      "type": "double",
      "description": "K",
      "isRequired": true
    }
  ]
}


In [29]:
scoring_endpoint = 'scoring/rest/configuration/{id}/score'.format(id=model_config['id'])
scoring_payload = {"id": model_config['id'], "requestInputTable": [{"id": model_meta_InputTable["id"],
                                                    "requestInputRow": [{"input": [{"name": "Age", "value": "23.0"},
                                                                                   {"name": "Sex", "value": "F"},
                                                                                   {"name": "BP", "value": "HIGH"},
                                                                                   {"name": "Cholesterol", "value": "HIGH"},
                                                                                   {"name": "Na", "value": "0.792535"},
                                                                                   {"name": "K", "value": "0.031258"}
                                                                                  ]}]}]}

In [30]:
resp_score = requests.post('/'.join([service_url, scoring_endpoint]), json = scoring_payload, headers=service_header)
scoring_result = resp_score.json()
print(json.dumps(scoring_result, indent=2))

{
  "providedBy": "ai_drug_mlp_proba",
  "id": "91260370-e56f-459a-9fbc-d5bc997a86c5",
  "columnNames": {
    "name": [
      "Age",
      "Sex",
      "BP",
      "Cholesterol",
      "Na",
      "K",
      "$N-Drug",
      "$NC-Drug",
      "$NP-drugA",
      "$NP-drugB",
      "$NP-drugC",
      "$NP-drugX",
      "$NP-drugY"
    ]
  },
  "rowValues": [
    {
      "value": [
        {
          "value": "23"
        },
        {
          "value": "F"
        },
        {
          "value": "HIGH"
        },
        {
          "value": "HIGH"
        },
        {
          "value": "0.792535"
        },
        {
          "value": "0.031258"
        },
        {
          "value": "drugY"
        },
        {
          "value": "0.9999929901781122"
        },
        {
          "value": "6.88638806109765E-6"
        },
        {
          "value": "1.2039675699035683E-7"
        },
        {
          "value": "1.6709057789050303E-10"
        },
        {
          "value": "2.8

**Multiclass classification using SVM** without probabilities

In [31]:
[model_config] = [cm for cm in config_meta if cm['id'] == 'ai_drug_svm']
print(json.dumps(model_config, indent=2))

{
  "cfgSerial": "091edbc5c4e74be5000001686a7c7110a1fd",
  "state": "ACTIVE",
  "cacheSize": 1,
  "id": "ai_drug_svm",
  "modelReference": {
    "label": "LATEST",
    "resourcePath": "/aiopenscale/Drug1_svm.str",
    "id": "091edbc5c4e74be5000001686a7c7110a1f3"
  },
  "configurationStatus": {
    "statusCode": "INFORMATION",
    "message": "Started"
  }
}


**List model metadata**

In [32]:
meta_endpoint = 'scoring/rest/configuration/{id}/metadata'.format(id=model_config['id'])
print(json.dumps(meta_endpoint, indent=2))

"scoring/rest/configuration/ai_drug_svm/metadata"


In [33]:
resp_meta = requests.get('/'.join([service_url, meta_endpoint]), headers=service_header)
model_meta = resp_meta.json()
print(json.dumps(model_meta, indent=2))

{
  "metadataInputTable": [
    {
      "id": "id1JRRSIWAHWX",
      "name": "scoreInput",
      "metadataInputField": [
        {
          "name": "Age",
          "type": "long",
          "description": "Age",
          "isRequired": true
        },
        {
          "name": "Sex",
          "type": "string",
          "description": "Sex",
          "isRequired": true
        },
        {
          "name": "BP",
          "type": "string",
          "description": "BP",
          "isRequired": true
        },
        {
          "name": "Cholesterol",
          "type": "string",
          "description": "Cholesterol",
          "isRequired": true
        },
        {
          "name": "Na",
          "type": "double",
          "description": "Na",
          "isRequired": true
        },
        {
          "name": "K",
          "type": "double",
          "description": "K",
          "isRequired": true
        }
      ]
    }
  ],
  "metadataOutputField": [
    {
      "name"

**Score heart drug model**

In [34]:
[model_meta_InputTable] = model_meta['metadataInputTable']
print(json.dumps(model_meta_InputTable, indent=2))

{
  "id": "id1JRRSIWAHWX",
  "name": "scoreInput",
  "metadataInputField": [
    {
      "name": "Age",
      "type": "long",
      "description": "Age",
      "isRequired": true
    },
    {
      "name": "Sex",
      "type": "string",
      "description": "Sex",
      "isRequired": true
    },
    {
      "name": "BP",
      "type": "string",
      "description": "BP",
      "isRequired": true
    },
    {
      "name": "Cholesterol",
      "type": "string",
      "description": "Cholesterol",
      "isRequired": true
    },
    {
      "name": "Na",
      "type": "double",
      "description": "Na",
      "isRequired": true
    },
    {
      "name": "K",
      "type": "double",
      "description": "K",
      "isRequired": true
    }
  ]
}


In [35]:
scoring_endpoint = 'scoring/rest/configuration/{id}/score'.format(id=model_config['id'])
scoring_payload = {"id": model_config['id'], "requestInputTable": [{"id": model_meta_InputTable["id"],
                                                    "requestInputRow": [{"input": [{"name": "Age", "value": "23.0"},
                                                                                   {"name": "Sex", "value": "F"},
                                                                                   {"name": "BP", "value": "HIGH"},
                                                                                   {"name": "Cholesterol", "value": "HIGH"},
                                                                                   {"name": "Na", "value": "0.792535"},
                                                                                   {"name": "K", "value": "0.031258"}
                                                                                  ]}]}]}

In [36]:
resp_score = requests.post('/'.join([service_url, scoring_endpoint]), json = scoring_payload, headers=service_header)
scoring_result = resp_score.json()
print(json.dumps(scoring_result, indent=2))

{
  "providedBy": "ai_drug_svm",
  "id": "df456b19-84c2-43d3-b907-c15cb3b858c2",
  "columnNames": {
    "name": [
      "Age",
      "Sex",
      "BP",
      "Cholesterol",
      "Na",
      "K",
      "$S-Drug"
    ]
  },
  "rowValues": [
    {
      "value": [
        {
          "value": "23"
        },
        {
          "value": "F"
        },
        {
          "value": "HIGH"
        },
        {
          "value": "HIGH"
        },
        {
          "value": "0.792535"
        },
        {
          "value": "0.031258"
        },
        {
          "value": "drugY"
        }
      ]
    }
  ]
}


## House prices model

**Regression model using classification and regression tree**

In [37]:
[model_config] = [cm for cm in config_meta if cm['id'] == 'ai_housing_model_crt']
print(json.dumps(model_config, indent=2))

{
  "cfgSerial": "091edbc5c4e74be5000001686a7c71109a99",
  "state": "ACTIVE",
  "cacheSize": 1,
  "id": "ai_housing_model_crt",
  "modelReference": {
    "label": "LATEST",
    "resourcePath": "/aiopenscale/housing_model.str",
    "id": "091edbc5c4e74be5000001686a7c71109918"
  },
  "configurationStatus": {
    "statusCode": "INFORMATION",
    "message": "Started"
  }
}


**List model metadata**

In [38]:
meta_endpoint = 'scoring/rest/configuration/{id}/metadata'.format(id=model_config['id'])
print(json.dumps(meta_endpoint, indent=2))

"scoring/rest/configuration/ai_housing_model_crt/metadata"


In [39]:
resp_meta = requests.get('/'.join([service_url, meta_endpoint]), headers=service_header)
model_meta = resp_meta.json()
print(json.dumps(model_meta, indent=2))

{
  "metadataInputTable": [
    {
      "id": "id7JFTNZZMJQL",
      "name": "housing_data.csv",
      "metadataInputField": [
        {
          "name": "CRIM",
          "type": "double",
          "description": "CRIM",
          "isRequired": true
        },
        {
          "name": "ZN",
          "type": "double",
          "description": "ZN",
          "isRequired": true
        },
        {
          "name": "INDUS",
          "type": "double",
          "description": "INDUS",
          "isRequired": true
        },
        {
          "name": "CHAS",
          "type": "long",
          "description": "CHAS",
          "isRequired": true
        },
        {
          "name": "NOX",
          "type": "double",
          "description": "NOX",
          "isRequired": true
        },
        {
          "name": "RM",
          "type": "double",
          "description": "RM",
          "isRequired": true
        },
        {
          "name": "AGE",
          "type": "double"

**Score house prices model**

In [40]:
[model_meta_InputTable] = model_meta['metadataInputTable']
print(json.dumps(model_meta_InputTable, indent=2))

# Input tables
print('\n', [{'id': x['id'], 'name': x['name']} for x in model_meta['metadataInputTable']])

# First input table field names
print('\n', [fld['name'] for fld in model_meta['metadataInputTable'][0]['metadataInputField']])

{
  "id": "id7JFTNZZMJQL",
  "name": "housing_data.csv",
  "metadataInputField": [
    {
      "name": "CRIM",
      "type": "double",
      "description": "CRIM",
      "isRequired": true
    },
    {
      "name": "ZN",
      "type": "double",
      "description": "ZN",
      "isRequired": true
    },
    {
      "name": "INDUS",
      "type": "double",
      "description": "INDUS",
      "isRequired": true
    },
    {
      "name": "CHAS",
      "type": "long",
      "description": "CHAS",
      "isRequired": true
    },
    {
      "name": "NOX",
      "type": "double",
      "description": "NOX",
      "isRequired": true
    },
    {
      "name": "RM",
      "type": "double",
      "description": "RM",
      "isRequired": true
    },
    {
      "name": "AGE",
      "type": "double",
      "description": "AGE",
      "isRequired": true
    },
    {
      "name": "DIS",
      "type": "double",
      "description": "DIS",
      "isRequired": true
    },
    {
      "name": "RAD",


In [41]:
scoring_endpoint = 'scoring/rest/configuration/{id}/score'.format(id=model_config['id'])
scoring_payload = {"id": model_config['id'], "requestInputTable": [{"id": model_meta_InputTable["id"],
                                                    "requestInputRow": [{"input": [{"name": "CRIM", "value": "0.00632"},
                                                                                   {"name": "ZN", "value": "18.00"},
                                                                                   {"name": "INDUS", "value": "2.310"},
                                                                                   {"name": "CHAS", "value": "0"},
                                                                                   {"name": "NOX", "value": "0.5380"},
                                                                                   {"name": "RM", "value": "6.5750"},
                                                                                   {"name": "AGE", "value": "65.20"},
                                                                                   {"name": "DIS", "value": "4.0900"},
                                                                                   {"name": "RAD", "value": "1"},
                                                                                   {"name": "TAX", "value": "296.0"},
                                                                                   {"name": "PTRATIO", "value": "15.30"},
                                                                                   {"name": "Bk", "value": "1.2600"},
                                                                                   {"name": "LSTAT", "value": "4.98"},
                                                                                   {"name": "MEDV", "value": "-1"}
                                                                                  ]}]}]}

In [42]:
resp_score = requests.post('/'.join([service_url, scoring_endpoint]), json = scoring_payload, headers=service_header)
scoring_result = resp_score.json()
print(json.dumps(scoring_result, indent=2))

{
  "providedBy": "ai_housing_model_crt",
  "id": "e1d31e2e-f04d-4b35-9ef2-20c354de50b7",
  "columnNames": {
    "name": [
      "CRIM",
      "ZN",
      "INDUS",
      "CHAS",
      "NOX",
      "RM",
      "AGE",
      "DIS",
      "RAD",
      "TAX",
      "PTRATIO",
      "Bk",
      "LSTAT",
      "MEDV",
      "$R-MEDV"
    ]
  },
  "rowValues": [
    {
      "value": [
        {
          "value": "0.00632"
        },
        {
          "value": "18.0"
        },
        {
          "value": "2.31"
        },
        {
          "value": "0"
        },
        {
          "value": "0.538"
        },
        {
          "value": "6.575"
        },
        {
          "value": "65.2"
        },
        {
          "value": "4.09"
        },
        {
          "value": "1"
        },
        {
          "value": "296.0"
        },
        {
          "value": "15.3"
        },
        {
          "value": "1.26"
        },
        {
          "value": "4.98"
        },
        {